# Bloque 3

# Obtener producto del prompt del usuario para generar el embedding

In [7]:
!pip install langchain_google_genai
!pip install streamlit
!pip install langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.0/974.0 kB 6.2 MB/s eta 0:00:00a 0:00:01


In [8]:
from google.cloud import aiplatform_v1
import streamlit as st 
from google.cloud import bigquery
from langchain_google_vertexai.llms import VertexAI
from langchain.prompts import PromptTemplate
import vertexai
import os
from langchain_google_vertexai import VertexAIEmbeddings

### Definimos Gemini como nuestro LLM así como el template para extraer el producto del mensaje del usuario

In [9]:
llm = VertexAI(model_name="gemini-pro", temperature=0.1)

In [10]:
embedding_template = """Eres el primer eslabón de una cadena que conformará un bot ecommerce y necesito en este primer paso recoger únicamente la información referente al producto que nos da el usuario.

El objetivo es que a partir de tu output se generará un embedding para buscar este producto que pide el usuario en nuestra base de datos vectorial (por tanto no te inventes nada).

Ejemplo: "Quiero unas gafas de sol negras porque estamos en verano", tu output debe ser "Gafas de sol negras".

La pregunta introducida por el usuario es: "{question}"

Output para generar el embedding: """

In [11]:
prompt_embedding_template = PromptTemplate.from_template(embedding_template)

### Definimos la cadena que genera el template, pasa éste al LLM y obtiene su respuesta

In [12]:
embedding_chain = prompt_embedding_template | llm

In [13]:
prompt = "Hola, qué tal, quiero unos vaqueros azules de talla pequeña porque tengo una fiesta mañana"

In [14]:
prompt_embedding = embedding_chain.invoke({"question": prompt})
print(prompt_embedding)

Vaqueros azules talla pequeña


# Obtener producto más similar y recomendarlo

### Obtenemos el producto más similar (sql_result) como en el bloque2

#### VertexAIEmbeddings para generar embedding

In [15]:
import vertexai
from langchain_google_vertexai import VertexAIEmbeddings

# Obtención embeddings
embeddings = VertexAIEmbeddings(model_name="textembedding-gecko@001")   

#### Generamos embedding y hacemos la búsqueda de vecinos más cercanos a éste en nuestro índice

In [16]:
from google.cloud import aiplatform_v1

# Set variables for the current deployed index.
API_ENDPOINT="1173193209.europe-west1-1017560508533.vdb.vertexai.goog"
INDEX_ENDPOINT="projects/1017560508533/locations/europe-west1/indexEndpoints/3571398484969914368"
DEPLOYED_INDEX_ID="products_data_index_civica"

# Configure Vector Search client
client_options = {
  "api_endpoint": API_ENDPOINT
}
vector_search_client = aiplatform_v1.MatchServiceClient(
  client_options=client_options,
)

# Build FindNeighborsRequest object
datapoint = aiplatform_v1.IndexDatapoint(
  feature_vector=embeddings.embed_query(prompt_embedding)
)
query = aiplatform_v1.FindNeighborsRequest.Query(
  datapoint=datapoint,
  # The number of nearest neighbors to be retrieved
  neighbor_count=3
)
request = aiplatform_v1.FindNeighborsRequest(
  index_endpoint=INDEX_ENDPOINT,
  deployed_index_id=DEPLOYED_INDEX_ID,
  # Request can have multiple queries
  queries=[query],
  return_full_datapoint=False,
)

# Execute the request
response = vector_search_client.find_neighbors(request)

# Handle the response
print(response)

nearest_neighbors {
  neighbors {
    datapoint {
      datapoint_id: "7073"
      crowding_tag {
        crowding_attribute: "0"
      }
    }
    distance: 0.6680965423583984
  }
  neighbors {
    datapoint {
      datapoint_id: "6366"
      crowding_tag {
        crowding_attribute: "0"
      }
    }
    distance: 0.6601314544677734
  }
  neighbors {
    datapoint {
      datapoint_id: "6878"
      crowding_tag {
        crowding_attribute: "0"
      }
    }
    distance: 0.6522124409675598
  }
}



#### Obtenemos más información de Big Query

In [18]:
from google.cloud import bigquery

client = bigquery.Client()

sql = f"""
SELECT *
FROM master-424910.ecommerce.products
WHERE ID = {response.nearest_neighbors[0].neighbors[0].datapoint.datapoint_id}
;
"""

sql_result = client.query(sql).to_dataframe()
sql_result.head()

,id,cost,category,name,brand,retail_price,department,sku,distribution_center_id
0,7073,12.504,Shorts,Lowrider Juniors 2.5 Inch Shorts in Ice Blue b...,Hurley,24.0,Women,C7217B04FE11F374F9A6737901025606,1


### Definimos el template con el cual a partir del mensaje del usuario y el resultado más similar encontrado en BBDD obtendremos la recomendación del producto

In [19]:
template = """Eres un bot encargado de responder y asesorar sobre productos de
una tienda en linea (ecommerce). Por lo tanto, se respetuoso y trata cordialmente a las personas.
Si la pregunta suya no tiene nada que ver con asesoramiento de productos, dile que ese no es tu trabajo.

Pregunta: {question}

El resultado más similar encontrado en la base de datos de productos es el siguiente:
{most_similar}

Dile al cliente nuestra recomendación según lo que ha preguntado informándole de todo lo posible para que pueda decidirse en su compra como nombre del artículo o precio.
Responde en el mismo idioma en el que te pregunten.

Respuesta: """

In [20]:
prompt_template = PromptTemplate.from_template(template)

### Definimos la cadena que genera el template, pasa éste al LLM y obtiene su respuesta

In [21]:
chain = prompt_template | llm

In [22]:
bot_result = chain.invoke({"question": prompt, "most_similar": sql_result})
print(bot_result)

Hola, te recomiendo los **Lowrider Juniors 2.5 Inch Shorts in Ice Blue** de la marca Hurley. Son unos vaqueros azules de talla pequeña perfectos para una fiesta. 

Aquí tienes más información sobre el producto:

* **Precio:** 12.50 €
* **Categoría:** Shorts
* **Nombre:** Lowrider Juniors 2.5 Inch Shorts in Ice Blue
* **Marca:** Hurley
* **Talla:** Pequeña
* **Color:** Azul

Estos vaqueros son de tiro bajo y tienen una longitud de 2.5 pulgadas. Están hechos de un tejido de mezclilla suave y cómodo. Tienen un cierre de botón y cremallera, y dos bolsillos delanteros y dos traseros.

Creo que estos vaqueros son perfectos para tu fiesta. Son elegantes y cómodos, y te quedarán genial.

¿Te gustaría que te recomendara algún otro producto?
